# Package installation

Your first task this time is actually the installation of a package. Try running the cell below, it will probably fail with an *ModuleNotFoundError*. This is because you haven't installed the pybaseball-package yet with the plain installation of anaconda. Also it's not a package that's available via any conda channel, so you will have to use the standard-installer pip.

Run ```pip install pybaseball``` in your terminal and try importing the packages again

In [ ]:
import pybaseball
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Your own support vector machine

Today's practice will deal with support vector machines. Using radial basis functions(RBF) as kernel functions we will analyze everyones' favorite sport: baseball! Due to its popularity you are obviously already familiar with the fact, that the [strike zone](https://de.wikipedia.org/wiki/Strike_Zone), which the pitcher has to hit in order to be awarded a "strike" against the batter and will otherwise be punished with a "ball", is defined as a rectangle reaching from the player's knees to his chest above the home base. Because this is common knowledge we don't need to mention, that this definition varies from player to player as a result and also the umpire's calls will impact the shape of the real shape of the strike zone. We will build a support vector machine for our favorite players to determine the decision boundary for the judgment, if a pitch will be a strike or a ball.

The player, who we'll have a look at, is the 2017 rookie of the year, Aaron Judge, who's 2.01m tall and therefore one of the tallest players in the MLB. Let's see what information is available.

In [ ]:
judge_id = pybaseball.playerid_lookup("Judge","Aaron")
judge_stats = pybaseball.statcast_batter('2010-01-01', '2022-01-01', 592450)
print(judge_stats.columns)

### Data exploration

Well this might be a little too much data for now, even for us baseball enthusiasts. If you really want to know what all of these stats mean, I'll refer you to [this site](https://baseballsavant.mlb.com/csv-docs).

For now I'll go ahead and pick the relevant columns for now.

In [ ]:
judge_df = judge_stats[["plate_x", "plate_z", "type"]]
print(judge_df[100:106])

There we go, this is the horizontal(plate_x) and vertical(plate_z) position of the ball when it crosses home plate from the catcher's perspective and whether the pitch resulted in a strike(**S**), ball(**B**) or a successful hit(**X**), but wait... this was too easy. Something feels off.

What's that "mixed types"-error message in cell number 2 about? Why would I show you the columns 100 to 105 and not simply df.head()? Also if I print columns 100 to 105, why does the index in the output say 30 to 35?

Another problem might lure in the type-column. What datatype is it? Why might this be a problem and what would we rather have? 

Inspect the data yourself and figure out the problems

### Data preparation

Firstly there is a bunch of NaNs in our data. Secondly SVMs are good with numerical data, but not with categorical data as they want to represent the data as vectors.

So we want to map our data to 0s for balls and 1s for strikes. The **map-method** of dataframes comes in super handy for this kind of task. As we are mostly interested in the umpire's decision we will ignore the balls, which went into play. They will turn into NaNs, which we still have to deal with anyway because of the first problem.

Create a new column called *label* in the dataframe, which maps balls and strikes to their respective value.

Now there is a bunch of NaNs in our data. We should decided what we want to do with these values. If there is some other relevant data in these rows, we might want to replace them with some other value. But in our case we have no reason to keep these rows, so we can just drop them.

Clean the data, so that there are no NaNs left.

Let's have a look at how our data actually looks like. I prepared a plot-function for you. Call it(```my_function(my_argument1, my_argument2,...)```) with your dataframe(containing the mapped values in a column called *labels*) as the only argument

In [ ]:
def plot_data(df):
    balls = df[df.label == 0]
    strikes = df[df.label == 1]
    
    fig, ax = plt.subplots()
    ball_scatter = plt.scatter(balls.plate_x, balls.plate_z, c="b", alpha=0.2)
    strike_scatter = plt.scatter(strikes.plate_x, strikes.plate_z, c="r", alpha=0.2)
    plt.title("Strike zone data")
    plt.legend((ball_scatter, strike_scatter),
           ('Ball', 'Strike'),
           loc='upper right',
           ncol=2,
           fontsize=10)
    ax.set_ylim(0, 6)
    ax.set_xlim(-2, 2)
    plt.show()



We now have a clean and handy dataframe with all the data, that we're interested in. Let's get ready to feed it into our SVM.

Assign the part of the dataframes, which we will use to predict the label, to a new variable called *features*. Also assign the column with the labels to a new variable called *labels*.

Remind yourself, that you already had to assign a subframe in a previous exercise.

We want to test the quality of your model later on, so we will split our data in a training-set and a test-set again. Use the appropiate function a split your data. The test-set should contain 20% of the total data. Make sure, that the data was split correctly by having a look at the shape of the resulting matrices/arrays

### Model training

Now it's finally time to create our model, train it with our data, predict our test-set and evaluate the models accuracy.
Use an rbf-kernel, C = 2, and gamma = 2 for now.

Also print the confusion-matrix of your predictions as an assessment of the model-performance. Don't forget that labels you want to display are is the list ["Balls", "Strikes"].

As a final reward you get to plot the decision boundary. Call the ```plot_strike_zone```-function with your cleaned dataframe and your trained model.

In [ ]:
def plot_strike_zone(df, model):
    fig, ax = plt.subplots()
    plt.scatter(df.plate_x, df.plate_z, c=df.label, cmap=plt.cm.coolwarm, alpha=0.2)
    plt.title("Aaron Judge strike zone")
    ax.set_ylim(0, 6)
    ax.set_xlim(-2, 2)
    x_min, x_max = ax.get_xlim()
    y_min, y_max = ax.get_ylim()
    h = 0.04
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                        np.arange(y_min, y_max, h))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    ax.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.5)
    plt.show()
    


**Congratulations on building your own SVM model!**

If your thirst for baseball data is still not quenched(understandably), feel free to compare Judge's strike zone to that of a smaller player like Jose Altuve.